In [43]:
import open3d as o3d
import numpy as np
from scipy.spatial import cKDTree

In [46]:
def chamfer_distance_kdtree(points1, points2):
    """
    Compute Chamfer Distance between two point clouds using KDTree.

    Args:
        points1: numpy array of shape (N, 3)
        points2: numpy array of shape (M, 3)

    Returns:
        chamfer_dist: scalar float
    """
    # pcd1 = o3d.io.read_point_cloud(points1)
    # pcd2 = o3d.io.read_point_cloud(points2)

    # points1 = np.asarray(points1)
    # points2 = np.asarray(points2)

    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)

    dist1, _ = tree1.query(points2)
    dist2, _ = tree2.query(points1)

    chamfer_dist = np.mean(dist1 ** 2) + np.mean(dist2 ** 2)
    return chamfer_dist

In [62]:
pcd_gt  = o3d.io.read_point_cloud(r"cube.ply")
pcd_pred = o3d.io.read_point_cloud(r"scaled_cube_500.ply")

pcd_gt_pt = np.asarray(pcd_gt.points)
pcd_pred_pt = np.asarray(pcd_pred.points)

cd = chamfer_distance_kdtree(pcd_gt_pt, pcd_pred_pt)
print("Chamfer Distance:", cd)

Chamfer Distance: 7071.5932884823505


In [63]:
# Reading point clouds
# pcd_gt  = o3d.io.read_point_cloud(r"scaled_output.ply")
# pcd_pred = o3d.io.read_point_cloud(r"scaled_cube_500.ply")

#colouring point clouds with colour
pcd_gt.paint_uniform_color([0, 0 , 1 ]) # blue colour
pcd_pred.paint_uniform_color([0, 1 , 0 ])  # green colour

# Compute centroids
center1 = pcd_gt .get_center()
center2 = pcd_pred.get_center()

# Move both to the origin
pcd_gt.translate(-center1)
pcd_pred.translate(-center2)

PointCloud with 500 points.

In [64]:
# finding the overlapping points
pcd_pred_tree = o3d.geometry.KDTreeFlann(pcd_pred)

gt_points = np.asarray(pcd_gt.points)
mask = np.zeros(len(gt_points), dtype=bool)

threshold =  10 # distance tolerance for overlap (adjust as per your scale)

for i, pt in enumerate(gt_points):
    [k, idx, dist] = pcd_pred_tree.search_knn_vector_3d(pt, 1)
    if dist[0] < threshold**2:
        mask[i] = True  # mark overlapped points


In [65]:
overlap_points = gt_points[mask]
non_overlap_points = gt_points[~mask]

# Create Open3D clouds
overlap_pcd = o3d.geometry.PointCloud()
overlap_pcd.points = o3d.utility.Vector3dVector(overlap_points)
overlap_pcd.paint_uniform_color([1, 0, 0])  # red = overlapped

non_overlap_pcd = o3d.geometry.PointCloud()
non_overlap_pcd.points = o3d.utility.Vector3dVector(non_overlap_points)
non_overlap_pcd.paint_uniform_color([0, 0, 0])  # black = non_overlapped


PointCloud with 500 points.

In [ ]:
# o3d.visualization.draw_geometries([overlap_pcd,non_overlap_pcd,pcd_pred])

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


In [67]:
num_points_gt = len(pcd_gt.points)
num_overlap_pt = len(overlap_points)
ratio_overlap = num_overlap_pt/num_points_gt

In [68]:
total_reward = -0.5 * cd + 1 * ratio_overlap

In [69]:
total_reward

np.float64(-3535.7966442411753)